In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-performance-dataset/Student_performance_data _.csv


In [15]:
pip install scikit-fuzzy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Step 1: Load the dataset
file_path = '/kaggle/input/students-performance-dataset/Student_performance_data _.csv'  # Update with your actual file path
data = pd.read_csv(file_path)

In [17]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   StudentID          2392 non-null   int64  
 1   Age                2392 non-null   int64  
 2   Gender             2392 non-null   int64  
 3   Ethnicity          2392 non-null   int64  
 4   ParentalEducation  2392 non-null   int64  
 5   StudyTimeWeekly    2392 non-null   float64
 6   Absences           2392 non-null   int64  
 7   Tutoring           2392 non-null   int64  
 8   ParentalSupport    2392 non-null   int64  
 9   Extracurricular    2392 non-null   int64  
 10  Sports             2392 non-null   int64  
 11  Music              2392 non-null   int64  
 12  Volunteering       2392 non-null   int64  
 13  GPA                2392 non-null   float64
 14  GradeClass         2392 non-null   float64
dtypes: float64(3), int64(12)
memory usage: 280.4 KB


In [21]:
# Step 2: Define fuzzy variables
gpa = ctrl.Antecedent(np.arange(0, 4.1, 0.1), 'GPA')
study_time = ctrl.Antecedent(np.arange(0, 21, 1), 'StudyTimeWeekly')
absences = ctrl.Antecedent(np.arange(0, 31, 1), 'Absences')
reward = ctrl.Consequent(np.arange(0, 101, 1), 'Reward')

# Define membership functions
gpa['low'] = fuzz.trapmf(gpa.universe, [0, 0, 2.0, 2.5])
gpa['medium'] = fuzz.trimf(gpa.universe, [2.0, 3.0, 3.5])
gpa['high'] = fuzz.trapmf(gpa.universe, [3.0, 3.5, 4.0, 4.0])

study_time['low'] = fuzz.trapmf(study_time.universe, [0, 0, 5, 6])
study_time['medium'] = fuzz.trimf(study_time.universe, [5, 10, 15])
study_time['high'] = fuzz.trapmf(study_time.universe, [12, 15, 20, 20])

absences['low'] = fuzz.trapmf(absences.universe, [0, 0, 3, 4])
absences['medium'] = fuzz.trimf(absences.universe, [3, 7, 11])
absences['high'] = fuzz.trapmf(absences.universe, [10, 15, 30, 30])

reward['low'] = fuzz.trapmf(reward.universe, [0, 0, 30, 50])
reward['medium'] = fuzz.trimf(reward.universe, [30, 60, 80])
reward['high'] = fuzz.trapmf(reward.universe, [70, 85, 100, 100])

# Step 3: Define fuzzy rules
rule1 = ctrl.Rule(gpa['high'] & study_time['high'], reward['high'])
rule2 = ctrl.Rule(gpa['medium'] & absences['medium'], reward['medium'])
rule3 = ctrl.Rule(gpa['low'] & absences['high'], reward['low'])

# Step 4: Create and simulate the fuzzy system
reward_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
reward_simulation = ctrl.ControlSystemSimulation(reward_ctrl)

# Iterate through the dataset and calculate rewards
for index, row in data.iterrows():
    # Input variables for the fuzzy system
    reward_simulation.input['GPA'] = row['GPA']  # Replace with column name for GPA
    reward_simulation.input['StudyTimeWeekly'] = row['StudyTimeWeekly']  # Replace with column name
    reward_simulation.input['Absences'] = row['Absences']  # Replace with column name
    
    # Perform fuzzy computation
    reward_simulation.compute()
    
    # Print the calculated reward
    print(f"Student {index + 1}: GPA = {row['GPA']}, Study Time Weekly = {row['StudyTimeWeekly']}, "
          f"Absences = {row['Absences']}, Reward = {reward_simulation.output['Reward']}")


Student 1: GPA = 2.929195591667681, Study Time Weekly = 19.833722807854716, Absences = 7.0, Reward = 56.65106075332941
Student 2: GPA = 3.042914833436377, Study Time Weekly = 15.40875605584674, Absences = 0.0, Reward = 85.31950866673566
Student 3: GPA = 0.1126022544661815, Study Time Weekly = 4.21056976881226, Absences = 26.0, Reward = 20.41666666666666
Student 4: GPA = 2.0542181397029484, Study Time Weekly = 10.028829473958217, Absences = 14.0, Reward = 21.25396825396824
Student 5: GPA = 1.2880611817953875, Study Time Weekly = 4.6724952729713305, Absences = 17.0, Reward = 20.41666666666666


KeyError: 'Reward'